In [5]:
from google.colab import files
uploaded = files.upload()

Saving qa_set.csv to qa_set.csv


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences as Pad
import re
import numpy as np
import json

In [7]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['qa_set.csv']))
df

,questions,answers
0,what character did natalie portman play in sta...,Padmé Amidala
1,what state does selena gomez?,New York City
2,what country is the grand bahama island in?,Bahamas
3,what character did john noble play in lord of ...,Denethor II
4,who does joakim noah play for?,Chicago Bulls
...,...,...
2829,what do you call russian currency?,Russian ruble
2830,where did sir donald bradman live?,Adelaide
2831,what is the name of the broncos mascot?,Miles
2832,what caused the russian financial crisis of 1998?,Allies of World War II


In [15]:
questions = df['questions'].to_list()
answers = df['answers'].to_list()

print(questions[10])
print(answers[10])

which countries border the us?
Canada


In [34]:
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r'[" "]+', " ", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [35]:
clean_questions = []
for question in questions: 
    clean_questions.append(clean_text(question))
    
clean_answers = []
for answer in answers:
    answer = 'startans ' + clean_text(answer) +' endans'
    clean_answers.append(answer)

print(clean_questions[10])
print(clean_answers[10])

which countries border the us
startans canada endans


In [36]:
print(len(clean_questions))
print(len(clean_answers))

2834
2834


In [37]:
minlen = 2
maxlen = 20
questions_temp = []
answers_temp = []

# filter the questions
for i,question in enumerate(clean_questions):
    if len(question.split()) <= maxlen and len(question.split()) >= minlen:
        questions_temp.append(question)
        answers_temp.append(clean_answers[i])
        
filtered_questions = []
filtered_answers = []

# filter the answers w.r.t qs
for i, answer in enumerate(answers_temp):
    if len(answer.split()) <= maxlen and len(answer.split()) >= minlen:
        filtered_answers.append(answer)
        filtered_questions.append(questions_temp[i])
        
print(len(filtered_answers))
print(len(filtered_questions))

2607
2607


In [38]:
print(filtered_questions[10])
print(filtered_answers[10])

which countries border the us
startans canada endans


In [22]:
vocabsize = 8000

# unk -> unknow vocab to replace not used vocabs
question_tokenizer = Tokenizer(num_words = vocabsize+1, oov_token = 'unk')
answer_tokenizer = Tokenizer(num_words = vocabsize+1,oov_token = 'unk')

# tokenize the questions and answers
question_tokenizer.fit_on_texts(filtered_questions)
answer_tokenizer.fit_on_texts(filtered_answers)

# build the input sequence and output sequence
q_sequences = question_tokenizer.texts_to_sequences(filtered_questions)
a_sequences = answer_tokenizer.texts_to_sequences(filtered_answers)

# pad sequences in same length so that we can train them in model
q_pad = Pad(q_sequences, padding = 'post')
a_pad = Pad(a_sequences, padding = 'post')

In [23]:
q_token_json = question_tokenizer.to_json()
a_token_json = answer_tokenizer.to_json()

In [26]:
with open('questions.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(q_token_json, ensure_ascii=False))
    f.close()

with open('answers.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(a_token_json, ensure_ascii=False))
    f.close()

np.savez('data.npz', q_pad, a_pad)

In [40]:
# save the data
from google.colab import files
files.download('questions.json')
files.download('answers.json')
files.download('data.npz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>